In [1]:
# Import necessary libraries
import pandas as pd
import folium

In [2]:
# Load the CSV file into a pandas DataFrame
file_path = 'LightrailStations_v2.csv'
df = pd.read_csv(file_path)

# Display the first few rows to verify the data is loaded correctly
df.head(10)

,X,Y,OBJECTID,PID,NAME,LOCAL_RTS,EXPRESS_RTS,LIMITED_RTS,REGIONAL_RTS,SKYRIDE_RTS,...,PARCEL_ID_NUMBER,DIAGRAM,LOCATION,RAIL_LINE,STOPID,STOP_LAT,STOP_LON,VMAP,CNRS,INFO_REPORT
0,-1.168745e+07,4.822727e+06,401,62,I-25 / Broadway Station,0-0L-11-14,,,NaN,NaN,...,Stop#: 23061 (South)<br />Stop#: 23041 (North),BroncosRide,I-25 & Broadway,D-E-H,33686,39.701235,-104.988860,NaN,NaN,
1,-1.168560e+07,4.830933e+06,396,57,30th / Downing Station,12-28-34,,,NaN,NaN,...,Stop#: 23051,NaN,Downing St & California St,L,33703,39.758977,-104.973786,NaN,NaN,NaN
2,-1.168779e+07,4.823698e+06,391,1,Alameda Station,1-3-4-52,,,NaN,NaN,...,Stop#: 23060 (South)<br />Stop#: 23042 (North),BroncosRide,Cherokee St & Alaska Pl,D-E-H,33704,39.708920,-104.992883,Platte Valley,PLFX,NaN
3,-1.167614e+07,4.808029e+06,398,2,Arapahoe at Village Center Station,66-153-169,,,NaN,AT,...,Stop#: 25990 (South)<br />Stop#: 34008 (North),BroncosRide,Caley & Yosemite,E-R,33705,39.601315,-104.887494,Arapahoe - Orchard,APFX,
4,-1.167792e+07,4.811985e+06,461,125,Belleview Station,73,,,NaN,NaN,...,Stop#: 25991 (South)<br />Stop#: 34010 (North),BroncosRide,Belleview & I-25,E-R,33706,39.627524,-104.904746,Belleview,BWFX,
5,-1.168161e+07,4.819498e+06,435,127,Colorado Station,21-40-46,,,NaN,NaN,...,Stop#: 25980 (South)<br />Stop#: 34013 (North),BroncosRide,Evans & I-25,E-H,33708,39.708879,-104.989493,NaN,NaN,NaN
6,-1.167433e+07,4.802470e+06,399,119,County Line Station,402L,,,NaN,NaN,...,Stop#: 25988 (South)<br />Stop#: 34006 (North),BroncosRide,County Line Rd & I-25 (Park Meadows Mall),E-R,33709,39.679398,-104.937673,Lone Tree - South Inverness,SIFX,
7,-1.167493e+07,4.814217e+06,440,130,Dayton Station,NaN,,,NaN,NaN,...,Stop#: 25974 (South)<br />Stop#: 34004 (North),NaN,I-225 & Yosemite,H-R,33710,39.561964,-104.870532,NaN,NaN,
8,-1.167483e+07,4.804960e+06,458,122,Dry Creek Station,NaN,,,NaN,NaN,...,Stop#: 25989 (South)<br />Stop#: 34007 (North),BroncosRide,Dry Creek & I-25,E-R,33711,39.643677,-104.878653,Dry Creek - North Inverness,DRFX - NIFX,NaN
9,-1.168855e+07,4.815999e+06,438,60,Englewood Station,0-12-35-51-TRLY,,,NaN,NaN,...,Stop#: 25436 (South)<br />Stop#: 24890 (North),BroncosRide,Floyd & Inca,D,33712,39.579487,-104.877416,NaN,NaN,


In [3]:
# Select the relevant columns
columns = ['NAME', 'LOCAL_RTS', 'EXPRESS_RTS', 'LIMITED_RTS', 'REGIONAL_RTS', 'SKYRIDE_RTS', 'SPECIAL_RTS',
           'ADDRESS', 'PARCEL_ID_NUMBER', 'LOCATION', 'RAIL_LINE', 'STOPID', 'STOP_LAT', 'STOP_LON']
df_filtered = df[columns]

# Drop rows with missing latitude or longitude data
df_filtered = df_filtered.dropna(subset=['STOP_LAT', 'STOP_LON'])

# Convert STOP_LAT and STOP_LON to numeric values (in case they're stored as strings)
df_filtered['STOP_LAT'] = pd.to_numeric(df_filtered['STOP_LAT'], errors='coerce')
df_filtered['STOP_LON'] = pd.to_numeric(df_filtered['STOP_LON'], errors='coerce')

# Display the filtered DataFrame
df_filtered.head(10)

,NAME,LOCAL_RTS,EXPRESS_RTS,LIMITED_RTS,REGIONAL_RTS,SKYRIDE_RTS,SPECIAL_RTS,ADDRESS,PARCEL_ID_NUMBER,LOCATION,RAIL_LINE,STOPID,STOP_LAT,STOP_LON
0,I-25 / Broadway Station,0-0L-11-14,,,NaN,NaN,D-E-H,901 S Broadway,Stop#: 23061 (South)<br />Stop#: 23041 (North),I-25 & Broadway,D-E-H,33686,39.701235,-104.988860
1,30th / Downing Station,12-28-34,,,NaN,NaN,L,2999 Downing St,Stop#: 23051,Downing St & California St,L,33703,39.758977,-104.973786
2,Alameda Station,1-3-4-52,,,NaN,NaN,D-E-H,499 S Cherokee St,Stop#: 23060 (South)<br />Stop#: 23042 (North),Cherokee St & Alaska Pl,D-E-H,33704,39.708920,-104.992883
3,Arapahoe at Village Center Station,66-153-169,,,NaN,AT,E-R,8800 E Caley Ave,Stop#: 25990 (South)<br />Stop#: 34008 (North),Caley & Yosemite,E-R,33705,39.601315,-104.887494
4,Belleview Station,73,,,NaN,NaN,E-R,4855 S Quebec St,Stop#: 25991 (South)<br />Stop#: 34010 (North),Belleview & I-25,E-R,33706,39.627524,-104.904746
5,Colorado Station,21-40-46,,,NaN,NaN,E-H,4300 E Colorado Center Dr,Stop#: 25980 (South)<br />Stop#: 34013 (North),Evans & I-25,E-H,33708,39.708879,-104.989493
6,County Line Station,402L,,,NaN,NaN,E-R,8340 S Valley Hwy,Stop#: 25988 (South)<br />Stop#: 34006 (North),County Line Rd & I-25 (Park Meadows Mall),E-R,33709,39.679398,-104.937673
7,Dayton Station,NaN,,,NaN,NaN,H-R,4151 S Dallas St,Stop#: 25974 (South)<br />Stop#: 34004 (North),I-225 & Yosemite,H-R,33710,39.561964,-104.870532
8,Dry Creek Station,NaN,,,NaN,NaN,E-R,9450 E Dry Creek Rd,Stop#: 25989 (South)<br />Stop#: 34007 (North),Dry Creek & I-25,E-R,33711,39.643677,-104.878653
9,Englewood Station,0-12-35-51-TRLY,,,NaN,NaN,D,899 W Floyd Ave,Stop#: 25436 (South)<br />Stop#: 24890 (North),Floyd & Inca,D,33712,39.579487,-104.877416


In [4]:
# Create a Folium map centered on Denver, Colorado
denver_map = folium.Map(location=[39.7392, -104.9903], zoom_start=12)  # Coordinates of Denver

# Add markers for each light rail station
for idx, row in df_filtered.iterrows():
    # Extract station information
    name = row['NAME']
    local_rts = row['LOCAL_RTS']
    express_rts = row['EXPRESS_RTS']
    limited_rts = row['LIMITED_RTS']
    regional_rts = row['REGIONAL_RTS']
    skyride_rts = row['SKYRIDE_RTS']
    special_rts = row['SPECIAL_RTS']
    address = row['ADDRESS']
    parcel_id = row['PARCEL_ID_NUMBER']
    location = row['LOCATION']
    rail_line = row['RAIL_LINE']
    stop_id = row['STOPID']
    lat = row['STOP_LAT']
    lon = row['STOP_LON']
    
    # Create the popup content with station details
    popup_content = (
        f"""
        <h3>Light Rail Information</h3>
        <b>Name:</b> {name}<br>
        <b>Local Routes:</b> {local_rts}<br>
        <b>Express Routes:</b> {express_rts}<br>
        <b>Limited Routes:</b> {limited_rts}<br>
        <b>Regional Routes:</b> {regional_rts}<br>
        <b>SkyRide Routes:</b> {skyride_rts}<br>
        <b>Special Routes:</b> {special_rts}<br>
        <b>Address:</b> {address}<br>
        <b>Parcel ID:</b> {parcel_id}<br>
        <b>Location:</b> {location}<br>
        <b>Rail Line:</b> {rail_line}<br>
        <b>Stop ID:</b> {stop_id}
        """
    )
    
    # Add a marker for each station
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_content, max_width=300),
        icon=folium.Icon(color='green', icon='train', prefix='fa')
    ).add_to(denver_map)

# Display the map
denver_map

In [5]:
# Step 7: Save the map to an HTML file
denver_map.save("denver_lightrail_stations_map.html")